# Sistemas de recomendación con LightFM
En este notebook se obtienen los distintos tipos de modelos (colaborativo, basado en contenido e híbrido) con el conjunto de datos de MovieLens y la librería LightFM.

In [1]:
# Importar todo lo necesario
import multiprocessing
import pickle
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import reciprocal_rank
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split

d:\anaconda\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
# Obtención del número de hilos del procesador
cpu_threads = multiprocessing.cpu_count()

## MovieLens

### Obtención de los dataframes

In [3]:
# Obtención los dataframes previamente creados
%store -r ml_data_df
%store -r ml_users_df
%store -r ml_items_df

### Obtención del dataset y de las matrices   
Convierto los dataframes en las estructuras de datos que necesita LightFM para poder sacar las matrices y poder hacer uso de su sistema de recomendación

In [4]:
# Obtención de los dataset
ml_dataset = Dataset()
ml_dataset.fit(ml_data_df['Id Usuario'], ml_data_df['Id Película'])
ml_dataset.fit_partial(users=ml_users_df['Id Usuario'], items=ml_items_df['Id Película'],
                    user_features=ml_users_df['Género'], item_features=ml_items_df['Título'])

# Obtención de las matrices
(ml_interactions, ml_weights) = ml_dataset.build_interactions((row['Id Usuario'], row['Id Película'], row['Valoración']) for index, row in ml_data_df.iterrows())
ml_item_features = ml_dataset.build_item_features((row['Id Película'], [row['Título']]) for index, row in ml_items_df.iterrows())
ml_user_features = ml_dataset.build_user_features((row['Id Usuario'], [row['Género']]) for index, row in ml_users_df.iterrows())

### División de los datos

In [5]:
# Divido el conjunto de datos en train y test
ml_train, ml_test = random_train_test_split(ml_interactions, test_percentage=0.2)

### Guardado de los datos

In [6]:
# Guardo los datos en un archivo pickle
ml_data_pickle = open('ml_data.pickle', 'wb')
pickle.dump([ml_train, ml_test, ml_item_features, ml_user_features], ml_data_pickle, protocol=pickle.HIGHEST_PROTOCOL)
ml_data_pickle.close()

In [7]:
# Recojo los datos
"""
ml_data_pickle = open('ml_data.pickle', 'rb')
[ml_train, ml_test, ml_item_features, ml_user_features] = pickle.load(ml_data_pickle)
"""

"\nml_data_pickle = open('ml_data.pickle', 'rb')\n[ml_train, ml_test, ml_item_features, ml_user_features] = pickle.load(ml_data_pickle)\n"

### Obtención de los modelos

#### Modelo colaborativo

In [8]:
# Obtención el modelo y entrenamiento
ml_collab_model = LightFM(loss='warp')
ml_collab_model.fit(ml_train, epochs=30, num_threads=cpu_threads)

In [9]:
# Obtención de la precisión k con el modelo colaborativo
ml_collab_precision_at_k = precision_at_k(ml_collab_model, ml_test, 
                                          train_interactions=ml_train, k=10, num_threads=cpu_threads).mean()

In [10]:
# Obtención del AUC score con el modelo colaborativo
ml_collab_auc_score = auc_score(ml_collab_model, ml_test, train_interactions=ml_train, num_threads=cpu_threads).mean()

In [11]:
# Obtención del recall k con el modelo colaborativo
ml_collab_recall_at_k = recall_at_k(ml_collab_model, ml_test, train_interactions=ml_train, k=10, num_threads=cpu_threads).mean()

In [12]:
# Obtención del reciprocal rank con el modelo colaborativo
ml_collab_reciprocal_rank = reciprocal_rank(ml_collab_model, ml_test, train_interactions=ml_train, 
                                            num_threads=cpu_threads).mean()

In [14]:
# Obtención del ranking para todos los usuarios/items
ml_collab_predictions = ml_collab_model.predict_rank(ml_test, 
                                                     train_interactions=ml_train, 
                                                     num_threads=cpu_threads,check_intersections=False)

#### Modelo híbrido

In [15]:
# Obtención el modelo y entrenamiento
ml_hybrid_model = LightFM(loss='warp')
ml_hybrid_model.fit(ml_train, item_features=ml_item_features, epochs=30, num_threads=cpu_threads)

In [16]:
# Obtención de la precisión k con el modelo híbrido
ml_hybrid_precision_at_k = precision_at_k(ml_hybrid_model, ml_test, train_interactions=ml_train, 
                                          item_features=ml_item_features, k=10, num_threads=cpu_threads).mean()

In [17]:
# Obtención del AUC score con el modelo híbrido
ml_hybrid_auc_score = auc_score(ml_hybrid_model, ml_test, train_interactions=ml_train, 
                                item_features=ml_item_features, num_threads=cpu_threads).mean()

In [18]:
# Obtención del recall k con el modelo híbrido
ml_hybrid_recall_at_k = recall_at_k(ml_hybrid_model, ml_test, train_interactions=ml_train, item_features=ml_item_features, 
                                    k=10, num_threads=cpu_threads).mean()

In [19]:
# Obtención del reciprocal rank con el modelo híbrido
ml_hybrid_reciprocal_rank = reciprocal_rank(ml_hybrid_model, ml_test, train_interactions=ml_train, 
                                            item_features=ml_item_features, num_threads=cpu_threads).mean()

In [21]:
# Obtención del ranking para todos los usuarios/items
ml_hybrid_predictions = ml_hybrid_model.predict_rank(ml_test, train_interactions=ml_train, 
                                                     item_features=ml_item_features,  
                                                     num_threads=multiprocessing.cpu_count())

#### Modelo por contenido

In [22]:
# Obtención el modelo y entrenamiento
ml_content_model = LightFM(loss='warp')
ml_content_model.fit(ml_train, user_features=ml_user_features, item_features=ml_item_features, 
                     epochs=30, num_threads=cpu_threads)

In [23]:
# Obtención de la precisión k con el modelo por contenido
ml_content_precision_at_k = precision_at_k(ml_content_model, ml_test, train_interactions=ml_train, 
                                           user_features=ml_user_features, 
                                           item_features=ml_item_features, k=10, num_threads=cpu_threads).mean()

In [24]:
# Obtención del AUC score con el modelo por contenido
ml_content_auc_score = auc_score(ml_content_model, ml_test, train_interactions=ml_train, 
                                 user_features=ml_user_features, 
                                 item_features=ml_item_features, num_threads=cpu_threads).mean()

In [25]:
# Obtención del recall k con el modelo por contenido
ml_content_recall_at_k = recall_at_k(ml_content_model, ml_test, train_interactions=ml_train, user_features=ml_user_features, 
                                     item_features=ml_item_features, k=10, num_threads=cpu_threads).mean()

In [26]:
# Obtención del reciprocal rank con el modelo por contenido
ml_content_reciprocal_rank = reciprocal_rank(ml_content_model, ml_test, train_interactions=ml_train, 
                                             user_features=ml_user_features, item_features=ml_item_features, 
                                             num_threads=cpu_threads).mean()

In [27]:
# Obtención del ranking para todos los usuarios/items
ml_content_predictions = ml_content_model.predict_rank(ml_test, train_interactions=ml_train, 
                                                       item_features=ml_item_features, 
                                                       user_features=ml_user_features, 
                                                       num_threads=cpu_threads)

### Guardado de los modelos y resultados

In [28]:
# Guardo los modelos en un archivo pickle
ml_models_pickle = open('ml_models.pickle', 'wb')
pickle.dump([ml_collab_model, ml_hybrid_model, ml_content_model], ml_models_pickle, protocol=pickle.HIGHEST_PROTOCOL)
ml_models_pickle.close()

In [29]:
# Recojo los modelos
"""
ml_models_pickle = open('ml_models.pickle', 'rb')
[ml_collab_model, ml_hybrid_model, ml_content_model] = pickle.load(ml_models_pickle)
"""

"\nml_models_pickle = open('ml_models.pickle', 'rb')\n[ml_collab_model, ml_hybrid_model, ml_content_model] = pickle.load(ml_models_pickle)\n"

In [30]:
# Guardo los resultados en un archivo pickle
ml_results_pickle = open('ml_results.pickle', 'wb')
pickle.dump([ml_collab_precision_at_k, ml_collab_auc_score, ml_collab_recall_at_k, ml_collab_reciprocal_rank, ml_collab_predictions, 
             ml_hybrid_precision_at_k, ml_hybrid_auc_score, ml_hybrid_recall_at_k, ml_hybrid_reciprocal_rank, ml_hybrid_predictions,
             ml_content_precision_at_k, ml_content_auc_score, ml_content_recall_at_k, ml_content_reciprocal_rank, ml_content_predictions], 
            ml_results_pickle, protocol=pickle.HIGHEST_PROTOCOL)
ml_results_pickle.close()

## Anime

### Obtención de los dataframes

In [32]:
# Obtención los dataframes previamente creados
%store -r anime_data_df
%store -r anime_items_df

### Obtención del dataset y de las matrices   
Convierto los dataframes en las estructuras de datos que necesita LightFM para poder sacar las matrices y poder hacer uso de su sistema de recomendación

In [33]:
# Obtención de los dataset
anime_dataset = Dataset()
anime_dataset.fit(anime_data_df['Id Usuario'], anime_data_df['Id Anime'])
anime_dataset.fit_partial(items=anime_items_df['Id Anime'], item_features=anime_items_df['Título'])

# Obtención de las matrices
(anime_interactions, anime_weights) = anime_dataset.build_interactions((row['Id Usuario'], row['Id Anime'], row['Valoración']) for index, row in anime_data_df.iterrows())
anime_item_features = anime_dataset.build_item_features((row['Id Anime'], [row['Título']]) for index, row in anime_items_df.iterrows())

### División de los datos

In [34]:
# Divido el conjunto de datos en train y test
anime_train, anime_test = random_train_test_split(anime_interactions, test_percentage=0.2)

### Guardado de los datos

In [36]:
# Guardo los datos en un archivo pickle
anime_data_pickle = open('anime_data.pickle', 'wb')
pickle.dump([anime_train, anime_test, anime_item_features], anime_data_pickle, protocol=pickle.HIGHEST_PROTOCOL)
anime_data_pickle.close()

In [37]:
# Recojo los datos
"""
anime_data_pickle = open('anime_data.pickle', 'rb')
[anime_train, anime_test, anime_item_features, anime_user_features] = pickle.load(anime_data_pickle)
"""

"\nanime_data_pickle = open('anime_data.pickle', 'rb')\n[anime_train, anime_test, anime_item_features, anime_user_features] = pickle.load(anime_data_pickle)\n"

### Obtención de los modelos

#### Modelo colaborativo

In [38]:
# Obtención el modelo y entrenamiento
anime_collab_model = LightFM(loss='warp')
anime_collab_model.fit(anime_train, epochs=30, num_threads=cpu_threads)

In [39]:
# Obtención de la precisión k con el modelo colaborativo
anime_collab_precision_at_k = precision_at_k(anime_collab_model, anime_test, 
                                             train_interactions=anime_train, k=10, num_threads=cpu_threads, 
                                             check_intersections=False).mean()

In [40]:
# Obtención del AUC score con el modelo colaborativo
anime_collab_auc_score = auc_score(anime_collab_model, anime_test, 
                                   train_interactions=anime_train, num_threads=cpu_threads, check_intersections=False).mean()

In [42]:
# Obtención del recall k con el modelo colaborativo
anime_collab_recall_at_k = recall_at_k(anime_collab_model, anime_test, 
                                       train_interactions=anime_train, k=10, num_threads=cpu_threads, 
                                       check_intersections=False).mean()

In [43]:
# Obtención del reciprocal rank con el modelo colaborativo
anime_collab_reciprocal_rank = reciprocal_rank(anime_collab_model, anime_test, train_interactions=anime_train, 
                                               num_threads=cpu_threads, check_intersections=False).mean()

In [44]:
# Obtención del ranking para todos los usuarios/items
anime_collab_predictions = anime_collab_model.predict_rank(anime_test, 
                                                           train_interactions=anime_train, 
                                                           num_threads=cpu_threads,
                                                           check_intersections=False)

#### Modelo híbrido

In [45]:
# Obtención el modelo y entrenamiento
anime_hybrid_model = LightFM(loss='warp')
anime_hybrid_model.fit(anime_interactions, item_features=anime_item_features, epochs=30, num_threads=cpu_threads)

In [46]:
# Obtención de la precisión k con el modelo híbrido
anime_hybrid_precision_at_k = precision_at_k(anime_hybrid_model, anime_test, train_interactions=anime_train, 
                                             item_features=anime_item_features, k=10, num_threads=cpu_threads, 
                                             check_intersections=False).mean()

In [47]:
# Obtención del AUC score con el modelo híbrido
anime_hybrid_auc_score = auc_score(anime_hybrid_model, anime_test, train_interactions=anime_train, 
                                item_features=anime_item_features, num_threads=cpu_threads, check_intersections=False).mean()

In [49]:
# Obtención del recall k con el modelo híbrido
anime_hybrid_recall_at_k = recall_at_k(anime_hybrid_model, anime_test, train_interactions=anime_train, 
                                       item_features=anime_item_features, k=10, num_threads=cpu_threads, 
                                       check_intersections=False).mean()

In [51]:
# Obtención del reciprocal rank con el modelo híbrido
anime_hybrid_reciprocal_rank = reciprocal_rank(anime_hybrid_model, anime_test, train_interactions=anime_train, 
                                               item_features=anime_item_features, num_threads=cpu_threads, 
                                               check_intersections=False).mean()

In [52]:
# Obtención del ranking para todos los usuarios/items
anime_hybrid_predictions = anime_hybrid_model.predict_rank(anime_test, 
                                                           train_interactions=anime_train, 
                                                           item_features=anime_item_features, 
                                                           num_threads=cpu_threads,
                                                           check_intersections=False)

### Guardado de los modelos y resultados

In [53]:
# Guardo los modelos en un archivo pickle
anime_models_pickle = open('anime_models.pickle', 'wb')
pickle.dump([anime_collab_model, anime_hybrid_model], anime_models_pickle, protocol=pickle.HIGHEST_PROTOCOL)
anime_models_pickle.close()

In [54]:
# Recojo los modelos
"""
anime_models_pickle = open('anime_models.pickle', 'rb')
[anime_collab_model, anime_hybrid_model] = pickle.load(anime_models_pickle)
"""

"\nanime_models_pickle = open('anime_models.pickle', 'rb')\n[anime_collab_model, anime_hybrid_model] = pickle.load(anime_models_pickle)\n"

In [55]:
# Guardo los resultados en un archivo pickle
anime_results_pickle = open('anime_results.pickle', 'wb')
pickle.dump([anime_collab_precision_at_k, anime_collab_auc_score, anime_collab_recall_at_k, anime_collab_reciprocal_rank, anime_collab_predictions, 
             anime_hybrid_precision_at_k, anime_hybrid_auc_score, anime_hybrid_recall_at_k, anime_hybrid_reciprocal_rank, anime_hybrid_predictions],
            anime_results_pickle, protocol=pickle.HIGHEST_PROTOCOL)
anime_results_pickle.close()

## Book-Crossing

### Obtención de los dataframes

In [56]:
# Obtención los dataframes previamente creados
%store -r bc_data_df
%store -r bc_users_df
%store -r bc_items_df

### Obtención del dataset y de las matrices   
Convierto los dataframes en las estructuras de datos que necesita LightFM para poder sacar las matrices y poder hacer uso de su sistema de recomendación

In [57]:
# Obtención de los dataset
bc_dataset = Dataset()
bc_dataset.fit(bc_data_df['Id Usuario'], bc_data_df['ISBN'])
bc_dataset.fit_partial(users=bc_users_df['Id Usuario'], items=bc_items_df['ISBN'],
                    user_features=bc_users_df['Edad'], item_features=bc_items_df['Título'])

#num_users, num_items = bc_dataset.interactions_shape()
#print('Num users: {}, num_items {}.'.format(num_users, num_items))

# Obtención de las matrices
(bc_interactions, bc_weights) = bc_dataset.build_interactions((row['Id Usuario'], row['ISBN'], row['Valoración']) for index, row in bc_data_df.iterrows())
bc_item_features = bc_dataset.build_item_features((row['ISBN'], [row['Título']]) for index, row in bc_items_df.iterrows())
bc_user_features = bc_dataset.build_user_features((row['Id Usuario'], [row['Edad']]) for index, row in bc_users_df.iterrows())

### División de los datos

In [58]:
# Divido el conjunto de datos en train y test
bc_train, bc_test = random_train_test_split(bc_interactions, test_percentage=0.2)

### Guardado de los datos

In [59]:
# Guardo los datos en un archivo pickle
bc_data_pickle = open('bc_data.pickle', 'wb')
pickle.dump([bc_train, bc_test, bc_item_features, bc_user_features], bc_data_pickle, protocol=pickle.HIGHEST_PROTOCOL)
bc_data_pickle.close()

In [60]:
# Recojo los datos
"""
bc_data_pickle = open('bc_data.pickle', 'rb')
[bc_train, bc_test, bc_item_features, bc_user_features] = pickle.load(bc_data_pickle)
"""

"\nbc_data_pickle = open('bc_data.pickle', 'rb')\n[bc_train, bc_test, bc_item_features, bc_user_features] = pickle.load(bc_data_pickle)\n"

### Obtención de los modelos

#### Modelo colaborativo

In [61]:
# Obtención el modelo y entrenamiento
bc_collab_model = LightFM(loss='warp')
bc_collab_model.fit(bc_train, epochs=30, num_threads=cpu_threads)

In [62]:
# Obtención de la precisión k con el modelo colaborativo
bc_collab_precision_at_k = precision_at_k(bc_collab_model, bc_test, 
                                          train_interactions=bc_train, k=10, num_threads=cpu_threads).mean()

In [63]:
# Obtención del AUC score con el modelo colaborativo
bc_collab_auc_score = auc_score(bc_collab_model, bc_test, 
                                train_interactions=bc_train, num_threads=cpu_threads).mean()

In [64]:
# Obtención del recall k con el modelo colaborativo
bc_collab_recall_at_k = recall_at_k(bc_collab_model, bc_test, train_interactions=bc_train, k=10, num_threads=cpu_threads).mean()

In [65]:
# Obtención del reciprocal rank con el modelo colaborativo
bc_collab_reciprocal_rank = reciprocal_rank(bc_collab_model, bc_test, train_interactions=bc_train, 
                                            num_threads=cpu_threads).mean()

In [66]:
# Obtención del ranking para todos los usuarios/items
bc_collab_predictions = bc_collab_model.predict_rank(bc_test, 
                                                     train_interactions=bc_train, 
                                                     num_threads=cpu_threads)

#### Modelo híbrido

In [67]:
# Obtención el modelo y entrenamiento
bc_hybrid_model = LightFM(loss='warp')
bc_hybrid_model.fit(bc_train, item_features=bc_item_features, epochs=30, num_threads=cpu_threads)

In [68]:
# Obtención de la precisión k con el modelo híbrido
bc_hybrid_precision_at_k = precision_at_k(bc_hybrid_model, bc_test, train_interactions=bc_train, 
                                          item_features=bc_item_features, k=10, num_threads=cpu_threads).mean()

In [69]:
# Obtención del AUC score con el modelo híbrido
bc_hybrid_auc_score = auc_score(bc_hybrid_model, bc_test, train_interactions=bc_train, 
                                item_features=bc_item_features, num_threads=cpu_threads).mean()

In [70]:
# Obtención del recall k con el modelo híbrido
bc_hybrid_recall_at_k = recall_at_k(bc_hybrid_model, bc_test, train_interactions=bc_train, item_features=bc_item_features, 
                                    k=10, num_threads=cpu_threads).mean()

In [71]:
# Obtención del reciprocal rank con el modelo híbrido
bc_hybrid_reciprocal_rank = reciprocal_rank(bc_hybrid_model, bc_test, train_interactions=bc_train, 
                                            item_features=bc_item_features, num_threads=cpu_threads).mean()

In [72]:
# Obtención del ranking para todos los usuarios/items
bc_hybrid_predictions = bc_hybrid_model.predict_rank(bc_test, 
                                                     train_interactions=bc_train, 
                                                     item_features=bc_item_features, 
                                                     num_threads=cpu_threads)

#### Modelo por contenido

In [73]:
# Obtención el modelo y entrenamiento
bc_content_model = LightFM(loss='warp')
bc_content_model.fit(bc_train, 
                     user_features=bc_user_features, 
                     item_features=bc_item_features, 
                     epochs=30, num_threads=cpu_threads)

In [74]:
# Obtención de la precisión k con el modelo por contenido
bc_content_precision_at_k = precision_at_k(bc_content_model, bc_test, train_interactions=bc_train, 
                                           user_features=bc_user_features,
                                           item_features=bc_item_features, k=10, num_threads=cpu_threads).mean()

In [75]:
# Obtención del AUC score con el modelo por contenido
bc_content_auc_score = auc_score(bc_content_model, bc_test, train_interactions=bc_train, 
                                 user_features=bc_user_features,
                                 item_features=bc_item_features, num_threads=cpu_threads).mean()

In [76]:
# Obtención del recall k con el modelo por contenido
bc_content_recall_at_k = recall_at_k(bc_content_model, bc_test, train_interactions=bc_train, user_features=bc_user_features, 
                                     item_features=bc_item_features, k=10, num_threads=cpu_threads).mean()

In [77]:
# Obtención del reciprocal rank con el modelo por contenido
bc_content_reciprocal_rank = reciprocal_rank(bc_content_model, bc_test, train_interactions=bc_train, 
                                             user_features=bc_user_features, item_features=bc_item_features, 
                                             num_threads=cpu_threads).mean()

In [78]:
# Obtención del ranking para todos los usuarios/items
bc_content_predictions = bc_content_model.predict_rank(bc_test, train_interactions=bc_train, 
                                                       item_features=bc_item_features, 
                                                       user_features=bc_user_features, 
                                                       num_threads=cpu_threads)

### Guardado de los modelos y resultados

In [79]:
# Guardo los modelos en un archivo pickle
bc_models_pickle = open('bc_models.pickle', 'wb')
pickle.dump([bc_collab_model, bc_hybrid_model, bc_content_model], bc_models_pickle, protocol=pickle.HIGHEST_PROTOCOL)
bc_models_pickle.close()

In [80]:
# Recojo los modelos
"""
bc_models_pickle = open('bc_models.pickle', 'rb')
[bc_collab_model, bc_hybrid_model, bc_content_model] = pickle.load(bc_models_pickle)
"""

"\nbc_models_pickle = open('bc_models.pickle', 'rb')\n[bc_collab_model, bc_hybrid_model, bc_content_model] = pickle.load(bc_models_pickle)\n"

In [81]:
# Guardo los resultados en un archivo pickle
bc_results_pickle = open('bc_results.pickle', 'wb')
pickle.dump([bc_collab_precision_at_k, bc_collab_auc_score, bc_collab_recall_at_k, bc_collab_reciprocal_rank, bc_collab_predictions, 
             bc_hybrid_precision_at_k, bc_hybrid_auc_score, bc_hybrid_recall_at_k, bc_hybrid_reciprocal_rank, bc_hybrid_predictions,
             bc_content_precision_at_k, bc_content_auc_score, bc_content_recall_at_k, bc_content_reciprocal_rank, bc_content_predictions], 
            bc_results_pickle, protocol=pickle.HIGHEST_PROTOCOL)
bc_results_pickle.close()

## LastFM

### Obtención de los dataframes

In [ ]:
# Obtención los dataframes previamente creados
%store -r lf_data_df
%store -r lf_users_df
%store -r lf_items_df

### Obtención del dataset y de las matrices   
Convierto los dataframes en las estructuras de datos que necesita LightFM para poder sacar las matrices y poder hacer uso de su sistema de recomendación

In [ ]:
# Obtención de los dataset
lf_dataset = Dataset()
lf_dataset.fit(lf_data_df['Id Usuario'], lf_data_df['Id Artista'])
lf_dataset.fit_partial(users=lf_users_df['Id Usuario'], items=lf_items_df['Id Artista'],
                    user_features=lf_users_df['Id Amigo'], item_features=lf_items_df['Nombre'])

# Obtención de las matrices
(lf_interactions, lf_weights) = lf_dataset.build_interactions((row['Id Usuario'], row['Id Artista'], row['Veces escuchado']) for index, row in lf_data_df.iterrows())
lf_item_features = lf_dataset.build_item_features((row['Id Artista'], [row['Nombre']]) for index, row in lf_items_df.iterrows())
lf_user_features = lf_dataset.build_user_features((row['Id Usuario'], [row['Id Amigo']]) for index, row in lf_users_df.iterrows())

### División de los datos

In [ ]:
# Divido el conjunto de datos en train y test
lf_train, lf_test = random_train_test_split(lf_interactions, test_percentage=0.2)

### Guardado de los datos

In [ ]:
# Guardo los datos en un archivo pickle
lf_data_pickle = open('lf_data.pickle', 'wb')
pickle.dump([lf_train, lf_test, lf_item_features, lf_user_features], lf_data_pickle, protocol=pickle.HIGHEST_PROTOCOL)
lf_data_pickle.close()

In [ ]:
# Recojo los datos
"""
lf_data_pickle = open('lf_data.pickle', 'rb')
[lf_train, lf_test, lf_item_features, lf_user_features] = pickle.load(lf_data_pickle)
"""

### Obtención de los modelos

#### Modelo colaborativo

In [ ]:
# Obtención el modelo y entrenamiento
lf_collab_model = LightFM(loss='warp')
lf_collab_model.fit(lf_train, epochs=30, num_threads=cpu_threads)

In [ ]:
# Obtención de la precisión k con el modelo colaborativo
lf_collab_precision_at_k = precision_at_k(lf_collab_model, lf_test, 
                                          train_interactions=lf_train, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del AUC score con el modelo colaborativo
lf_collab_auc_score = auc_score(lf_collab_model, lf_test, 
                                train_interactions=lf_train, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del recall k con el modelo colaborativo
lf_collab_recall_at_k = recall_at_k(lf_collab_model, lf_test, train_interactions=lf_train, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del reciprocal rank con el modelo colaborativo
lf_collab_reciprocal_rank = reciprocal_rank(lf_collab_model, lf_test, train_interactions=lf_train, 
                                            num_threads=cpu_threads).mean()

In [ ]:
# Obtención del ranking para todos los usuarios/items
lf_collab_predictions = lf_collab_model.predict_rank(lf_test, 
                                                     train_interactions=lf_train, 
                                                     num_threads=cpu_threads)

#### Modelo híbrido

In [ ]:
# Obtención el modelo y entrenamiento
lf_hybrid_model = LightFM(loss='warp')
lf_hybrid_model.fit(lf_train, item_features=lf_item_features, epochs=30, num_threads=cpu_threads)

In [ ]:
# Obtención de la precisión k con el modelo híbrido
lf_hybrid_precision_at_k = precision_at_k(lf_hybrid_model, lf_test, train_interactions=lf_train, 
                                          item_features=lf_item_features, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del AUC score con el modelo híbrido
lf_hybrid_auc_score = auc_score(lf_hybrid_model, lf_test, train_interactions=lf_train, 
                                item_features=lf_item_features, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del recall k con el modelo híbrido
lf_hybrid_recall_at_k = recall_at_k(lf_hybrid_model, lf_test, train_interactions=lf_train, item_features=lf_item_features, 
                                    k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del reciprocal rank con el modelo híbrido
lf_hybrid_reciprocal_rank = reciprocal_rank(lf_hybrid_model, lf_test, train_interactions=lf_train, 
                                            item_features=lf_item_features, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del ranking para todos los usuarios/items
lf_hybrid_predictions = lf_hybrid_model.predict_rank(lf_test,
                                                    train_interactions=lf_train,
                                                    item_features=lf_item_features,
                                                    num_threads=multiprocessing.cpu_count())

#### Modelo por contenido

In [ ]:
# Obtención el modelo y entrenamiento
lf_content_model = LightFM(loss='warp')
lf_content_model.fit(lf_train, 
                     user_features=lf_user_features, 
                     item_features=lf_item_features, 
                     epochs=30, num_threads=cpu_threads)

In [ ]:
# Obtención de la precisión k con el modelo por contenido
lf_content_precision_at_k = precision_at_k(lf_content_model, lf_test, train_interactions=lf_train, 
                                           user_features=lf_user_features,
                                           item_features=lf_item_features, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del AUC score con el modelo por contenido
lf_content_auc_score = auc_score(lf_content_model, lf_test, train_interactions=lf_train, 
                                 user_features=lf_user_features,
                                 item_features=lf_item_features, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del recall k con el modelo por contenido
lf_content_recall_at_k = recall_at_k(lf_content_model, lf_test, train_interactions=lf_train, user_features=lf_user_features, 
                                     item_features=lf_item_features, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del reciprocal rank con el modelo por contenido
lf_content_reciprocal_rank = reciprocal_rank(lf_content_model, lf_test, train_interactions=lf_train, 
                                             user_features=lf_user_features, item_features=lf_item_features, 
                                             num_threads=cpu_threads).mean()

In [ ]:
# Obtención del ranking para todos los usuarios/items
lf_content_predictions = lf_content_model.predict_rank(lf_test, train_interactions=lf_train, 
                                                       item_features=lf_item_features, 
                                                       user_features=lf_user_features, 
                                                       num_threads=cpu_threads)

### Guardado de los modelos y resultados

In [ ]:
# Guardo los modelos en un archivo pickle
lf_models_pickle = open('lf_models.pickle', 'wb')
pickle.dump([lf_collab_model, lf_hybrid_model, lf_content_model], lf_models_pickle, protocol=pickle.HIGHEST_PROTOCOL)
lf_models_pickle.close()

In [ ]:
# Recojo los modelos
"""
lf_models_pickle = open('lf_models.pickle', 'rb')
[lf_collab_model, lf_hybrid_model, lf_content_model] = pickle.load(lf_models_pickle)
"""

In [ ]:
# Guardo los resultados en un archivo pickle
lf_results_pickle = open('lf_results.pickle', 'wb')
pickle.dump([lf_collab_precision_at_k, lf_collab_auc_score, lf_collab_recall_at_k, lf_collab_reciprocal_rank, lf_collab_predictions, 
             lf_hybrid_precision_at_k, lf_hybrid_auc_score, lf_hybrid_recall_at_k, lf_hybrid_reciprocal_rank, lf_hybrid_predictions,
             lf_content_precision_at_k, lf_content_auc_score, lf_content_recall_at_k, lf_content_reciprocal_rank, lf_content_predictions], 
            lf_results_pickle, protocol=pickle.HIGHEST_PROTOCOL)
lf_results_pickle.close()

## Dating Agency

### Obtención de los dataframes

In [ ]:
# Obtención los dataframes previamente creados
%store -r dating_data_df
%store -r dating_users_df

### Obtención del dataset y de las matrices   
Convierto los dataframes en las estructuras de datos que necesita LightFM para poder sacar las matrices y poder hacer uso de su sistema de recomendación

In [ ]:
# Obtención de los dataset
dating_dataset = Dataset()
dating_dataset.fit(dating_data_df['Id Usuario'], dating_data_df['Id Match'])
dating_dataset.fit_partial(users=dating_users_df['Id Usuario'], items=dating_users_df['Id Usuario'],
                    user_features=dating_users_df['Género'], item_features=dating_users_df['Género'])

# Obtención de las matrices
(dating_interactions, dating_weights) = dating_dataset.build_interactions((row['Id Usuario'], row['Id Match'], row['Valoración']) for index, row in dating_data_df.iterrows())
dating_item_features = dating_dataset.build_item_features((row['Id Usuario'], [row['Género']]) for index, row in dating_users_df.iterrows())
dating_user_features = dating_dataset.build_user_features((row['Id Usuario'], [row['Género']]) for index, row in dating_users_df.iterrows())

### División de los datos

In [ ]:
# Divido el conjunto de datos en train y test
dating_train, dating_test = random_train_test_split(dating_interactions, test_percentage=0.2)

### Guardado de los datos

In [ ]:
# Guardo los datos en un archivo pickle
dating_data_pickle = open('dating_data.pickle', 'wb')
pickle.dump([dating_train, dating_test, dating_item_features, dating_user_features], 
            dating_data_pickle, protocol=pickle.HIGHEST_PROTOCOL)
dating_data_pickle.close()

In [ ]:
# Recojo los datos
"""
dating_data_pickle = open('dating_data.pickle', 'rb')
[dating_train, dating_test, dating_item_features, dating_user_features] = pickle.load(dating_data_pickle)
"""

### Obtención de los modelos

#### Modelo colaborativo

In [ ]:
# Obtención el modelo y entrenamiento
dating_collab_model = LightFM(loss='warp')
dating_collab_model.fit(dating_train, epochs=30, num_threads=cpu_threads)

In [ ]:
# Obtención de la precisión k con el modelo colaborativo
dating_collab_precision_at_k = precision_at_k(dating_collab_model, dating_test, 
                                              train_interactions=dating_train, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del AUC score con el modelo colaborativo
dating_collab_auc_score = auc_score(dating_collab_model, dating_test, 
                                    train_interactions=dating_train, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del recall k con el modelo colaborativo
dating_collab_recall_at_k = recall_at_k(dating_collab_model, dating_test, train_interactions=dating_train, 
                                        k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del reciprocal rank con el modelo colaborativo
dating_collab_reciprocal_rank = reciprocal_rank(dating_collab_model, dating_test, train_interactions=dating_train, 
                                                num_threads=cpu_threads).mean()

In [ ]:
# Obtención del ranking para todos los usuarios/items
dating_collab_predictions = dating_collab_model.predict_rank(dating_test, 
                                                             train_interactions=dating_train, 
                                                             num_threads=cpu_threads)

#### Modelo híbrido

In [ ]:
# Obtención el modelo y entrenamiento
dating_hybrid_model = LightFM(loss='warp')
dating_hybrid_model.fit(dating_train, item_features=dating_item_features, epochs=30, num_threads=cpu_threads)

In [ ]:
# Obtención de la precisión k con el modelo híbrido
dating_hybrid_precision_at_k = precision_at_k(dating_hybrid_model, dating_test, train_interactions=dating_train, 
                                              item_features=dating_item_features, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del AUC score con el modelo híbrido
dating_hybrid_auc_score = auc_score(dating_hybrid_model, dating_test, train_interactions=dating_train, 
                                    item_features=dating_item_features, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del recall k con el modelo híbrido
dating_hybrid_recall_at_k = recall_at_k(dating_hybrid_model, dating_test, train_interactions=dating_train, 
                                        item_features=dating_item_features, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del reciprocal rank con el modelo híbrido
dating_hybrid_reciprocal_rank = reciprocal_rank(dating_hybrid_model, dating_test, train_interactions=dating_train, 
                                                item_features=dating_item_features, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del ranking para todos los usuarios/items
dating_hybrid_predictions = dating_hybrid_model.predict_rank(dating_test, 
                                                             train_interactions=dating_train, 
                                                             item_features=dating_item_features, 
                                                             num_threads=cpu_threads)

#### Modelo por contenido

In [ ]:
# Obtención el modelo y entrenamiento
dating_content_model = LightFM(loss='warp')
dating_content_model.fit(dating_train, 
                         user_features=dating_user_features, 
                         item_features=dating_item_features, 
                         epochs=30, num_threads=cpu_threads)

In [ ]:
# Obtención de la precisión k con el modelo por contenido
dating_content_precision_at_k = precision_at_k(dating_content_model, dating_test, train_interactions=dating_train, 
                                               user_features=dating_user_features, 
                                               item_features=dating_item_features, k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del AUC score con el modelo por contenido
dating_content_auc_score = auc_score(dating_content_model, dating_test, train_interactions=dating_train, 
                                     user_features=dating_user_features, 
                                     item_features=dating_item_features, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del recall k con el modelo por contenido
dating_content_recall_at_k = recall_at_k(dating_content_model, dating_test, train_interactions=dating_train, 
                                         user_features=dating_user_features, item_features=dating_item_features, 
                                         k=10, num_threads=cpu_threads).mean()

In [ ]:
# Obtención del reciprocal rank con el modelo por contenido
dating_content_reciprocal_rank = reciprocal_rank(dating_content_model, dating_test, train_interactions=dating_train, 
                                                 user_features=dating_user_features, item_features=dating_item_features, 
                                                 num_threads=cpu_threads).mean()

In [ ]:
# Obtención del ranking para todos los usuarios/items
dating_content_predictions = dating_content_model.predict_rank(dating_test, 
                                                               train_interactions=dating_train, 
                                                               item_features=dating_item_features, 
                                                               user_features=dating_user_features, 
                                                               num_threads=cpu_threads)

### Guardado de los modelos y resultados

In [ ]:
# Guardo los modelos en un archivo pickle
dating_models_pickle = open('dating_models.pickle', 'wb')
pickle.dump([dating_collab_model, dating_hybrid_model, dating_content_model], dating_models_pickle, 
            protocol=pickle.HIGHEST_PROTOCOL)
dating_models_pickle.close()

In [ ]:
# Recojo los modelos
"""
dating_models_pickle = open('dating_models.pickle', 'rb')
[dating_collab_model, dating_hybrid_model, dating_content_model] = pickle.load(dating_models_pickle)
"""

In [ ]:
# Guardo los resultados en un archivo pickle
dating_results_pickle = open('dating_results.pickle', 'wb')
pickle.dump([dating_collab_precision_at_k, dating_collab_auc_score, dating_collab_recall_at_k, dating_collab_reciprocal_rank, dating_collab_predictions, 
             dating_hybrid_precision_at_k, dating_hybrid_auc_score, dating_hybrid_recall_at_k, dating_hybrid_reciprocal_rank, dating_hybrid_predictions,
             dating_content_precision_at_k, dating_content_auc_score, dating_content_recall_at_k, dating_content_reciprocal_rank, dating_content_predictions], 
            dating_results_pickle, protocol=pickle.HIGHEST_PROTOCOL)
dating_results_pickle.close()